In [1]:
import pandas as pd
import numpy as np
import json
import os
from pandas.io.json import json_normalize
import re

In [2]:
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [3]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [54]:
cl_id = '66dded11300a4c5a9d565000fa13d8c8'
cl_sec = '1b74966ace0a4560bd7164f323a1afa6'
username = 'antonstorozhilov'
redirect_uri = 'http://localhost'
scope = 'user-read-currently-playing user-read-recently-played user-library-read playlist-modify-public playlist-read-private'
token = util.prompt_for_user_token(username, scope, client_id=cl_id, client_secret=cl_sec, redirect_uri=redirect_uri)
                                   #redirect_uri=redirect_uri)
    

In [55]:
spotify = spotipy.Spotify(auth=token)

In [6]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.scdn.co/image/ab67616d00004851072e9faef2ef7b6db63834a3")

In [25]:
with open('fifty_played_export.json', 'r') as f:
    recent_50_played = json_normalize(json.loads(f.read()))[['track.id', 'played_at', 'track.name', 'track.artists', 'track.album.name', 'track.album.id', 'track.album.images']]

In [26]:
recent_50_played['track.artists'][17]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0NcPKaSNIHAM2RfioH9vMT'},
  'href': 'https://api.spotify.com/v1/artists/0NcPKaSNIHAM2RfioH9vMT',
  'id': '0NcPKaSNIHAM2RfioH9vMT',
  'name': 'Guapdad 4000',
  'type': 'artist',
  'uri': 'spotify:artist:0NcPKaSNIHAM2RfioH9vMT'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4AA474G2hRfrHyGrfyDseO'},
  'href': 'https://api.spotify.com/v1/artists/4AA474G2hRfrHyGrfyDseO',
  'id': '4AA474G2hRfrHyGrfyDseO',
  'name': 'Mozzy',
  'type': 'artist',
  'uri': 'spotify:artist:4AA474G2hRfrHyGrfyDseO'}]

In [27]:
recent_50_played['track.artists.names'] = recent_50_played['track.artists'].apply(lambda x: list(map(lambda y: y['name'], x)))

In [28]:
recent_50_played['track.artists.id'] = recent_50_played['track.artists'].apply(lambda x: list(map(lambda y: y['id'], x)))

In [30]:
recent_50_played=recent_50_played.drop(['track.artists'], axis=1)

In [57]:
recent_50_played

,track.id,played_at,track.name,track.album.name,track.album.id,track.album.images,track.artists.names,track.artists.id
0,11kDth1aKUEUMq9r1pqyds,2020-01-10T18:29:59.199Z,5% TINT,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
1,20MuVazoNMv6xjKPnRFOxG,2020-01-10T18:26:43.090Z,WAKE UP,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
2,6NMtzpDQBTOfJwMzgMX0zl,2020-01-10T18:22:51.162Z,SKELETONS,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
3,6OaVWaGfhXn70ZJfdVcabn,2020-01-10T18:20:25.591Z,NO BYSTANDERS,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
4,1xYpVRspgaLxdJLOTQkTjg,2020-01-10T18:16:47.306Z,STOP TRYING TO BE GOD,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
5,4SZepBIPDRwPaHIjAKwRDb,2020-01-10T18:10:04.407Z,R.I.P. SCREW,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
6,2xLMifQCjDGFmkHkpNLD9h,2020-01-10T18:06:58.794Z,SICKO MODE,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
7,7KZ5MMVgBVox9ycroB2UrI,2020-01-10T18:01:45.932Z,CAROUSEL,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
8,7wBJfHzpfI3032CSD7CE2m,2020-01-10T17:58:45.831Z,STARGAZING,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
9,1gIa5W0AhvgfKgCu2RNR8C,2020-01-10T17:52:53.519Z,The City,Once Upon a Crime,04fJPhZwJpdMf2rcdSJ3o4,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[The Godfathers, Necro, Kool G Rap]","[3r6Gv2tZAgMI6djGZO4Djq, 51vjIMmHWyVqDrkxD07Jh..."


In [66]:
spotify.audio_features('11kDth1aKUEUMq9r1pqyds')[0]

{'danceability': 0.712,
 'energy': 0.481,
 'key': 4,
 'loudness': -5.415,
 'mode': 0,
 'speechiness': 0.0487,
 'acousticness': 0.173,
 'instrumentalness': 0,
 'liveness': 0.241,
 'valence': 0.062,
 'tempo': 140.0,
 'type': 'audio_features',
 'id': '11kDth1aKUEUMq9r1pqyds',
 'uri': 'spotify:track:11kDth1aKUEUMq9r1pqyds',
 'track_href': 'https://api.spotify.com/v1/tracks/11kDth1aKUEUMq9r1pqyds',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/11kDth1aKUEUMq9r1pqyds',
 'duration_ms': 196138,
 'time_signature': 4}

In [68]:
song_features = pd.DataFrame()

In [75]:
for i, v in recent_50_played.iterrows():
    audio_feat = spotify.audio_features(v['track.id'])[0]
    audio_feat['track_id'] = v['track.id']
    audio_feat['track_name'] = v['track.name']
    audio_feat['track_artists'] = v['track.artists.names'] 
    
    genres_list = list()
    for i in v['track.artists.id']:
        genres_list.append(spotify.artist(i)['genres'])
    audio_feat['track_genres'] = genres_list
    
    song_features = song_features.append(audio_feat, ignore_index=True)

11kDth1aKUEUMq9r1pqyds
20MuVazoNMv6xjKPnRFOxG
6NMtzpDQBTOfJwMzgMX0zl
6OaVWaGfhXn70ZJfdVcabn
1xYpVRspgaLxdJLOTQkTjg
4SZepBIPDRwPaHIjAKwRDb
2xLMifQCjDGFmkHkpNLD9h
7KZ5MMVgBVox9ycroB2UrI
7wBJfHzpfI3032CSD7CE2m
1gIa5W0AhvgfKgCu2RNR8C
0UyKtU3KuPsuSIr96c4eTu
0ljcLS6EfiHj5GzpAY5WiQ
5ttZ1pvgyKJAQQHD1q3Gqb
1FOINGDWa0dPgCpMabLT4Y
7f2HHmhlbdkW5f9Ge26qny
7h0d2h0fUmzbs7zeFigJPn
40mjsnRjCpycdUw3xhS20g
72JmSHyBLDFE4odn9iHjyy
5MqPHMhLXyZDDPHMGEF6MQ
5MqPHMhLXyZDDPHMGEF6MQ
4NeQdvqUrSv7iSTci5ZZRI
5QqyRUZeBE04yJxsD1OC0I
2NU7b9zjOK1NAKtzF5WQxl
5HjXHLruir1Av0gpuiwVmq
0WZZJ0mDKIkLfL0710ssZt
66aoMO408FA4pLR2xWVH3f
5EhP08K5zqnX196GtSe4wW
14EUcXXpqrC03Dzzdj8bGr
1NZs6n6hl8UuMaX0UC0YTz
1NZs6n6hl8UuMaX0UC0YTz
1PNGJBI1Lx0h8QCpkHKZLU
37jTPJgwCCmIGMPB45jrPV
6rxEjkoar48SssZePbtb2x
6qlfzOpeLcdoSxrgbkR0Zi
7wS94SSkAxwYiCsauxuqWc
39CyGKIFqMzSuynOR8uilD
05lOXZuJyELJT0gS1cdyzj
7klsJBCrYh6iX2sYqz4ZaQ
30TGx1nKoGjiBk1Vou7a61
30LqUnUPnYN8oqyniSEral
1wHKgpL6wb7V922tOJCIl3
6nwAubYnLGl7F7cDuOoVLp
0UyKtU3KuPsuSIr96c4eTu
1FOINGDWa0d

In [78]:
song_features = song_features.drop(['analysis_url', 'track_href', 'uri', 'id', 'type'], axis=1)

In [82]:
song_features= song_features.set_index('track_id')

In [83]:
song_features

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_artists,track_genres,track_name,valence
track_id,,,,,,,,,,,,,,,,
11kDth1aKUEUMq9r1pqyds,0.17300,0.712,196138.0,0.4810,0.000000,4.0,0.2410,-5.415,0.0,0.0487,140.000,4.0,[Travis Scott],"[[pop rap, rap]]",5% TINT,0.0620
20MuVazoNMv6xjKPnRFOxG,0.19200,0.741,231964.0,0.6450,0.000000,11.0,0.1630,-5.116,0.0,0.0963,149.070,4.0,[Travis Scott],"[[pop rap, rap]]",WAKE UP,0.4880
6NMtzpDQBTOfJwMzgMX0zl,0.00146,0.460,145588.0,0.6860,0.000000,0.0,0.3750,-5.948,0.0,0.0367,148.054,4.0,[Travis Scott],"[[pop rap, rap]]",SKELETONS,0.2520
6OaVWaGfhXn70ZJfdVcabn,0.02040,0.731,218284.0,0.8110,0.000000,1.0,0.4110,-3.225,1.0,0.0712,147.045,4.0,[Travis Scott],"[[pop rap, rap]]",NO BYSTANDERS,0.4260
1xYpVRspgaLxdJLOTQkTjg,0.22100,0.588,338438.0,0.5670,0.000000,4.0,0.1350,-3.738,0.0,0.0628,150.009,4.0,[Travis Scott],"[[pop rap, rap]]",STOP TRYING TO BE GOD,0.1130
4SZepBIPDRwPaHIjAKwRDb,0.38900,0.607,185610.0,0.6090,0.000016,0.0,0.1110,-5.978,0.0,0.2910,74.946,4.0,[Travis Scott],"[[pop rap, rap]]",R.I.P. SCREW,0.5560
2xLMifQCjDGFmkHkpNLD9h,0.00513,0.834,312820.0,0.7300,0.000000,8.0,0.1240,-3.714,1.0,0.2220,155.008,4.0,[Travis Scott],"[[pop rap, rap]]",SICKO MODE,0.4460
11kDth1aKUEUMq9r1pqyds,0.17300,0.712,196138.0,0.4810,0.000000,4.0,0.2410,-5.415,0.0,0.0487,140.000,4.0,[Travis Scott],"[[pop rap, rap]]",5% TINT,0.0620
20MuVazoNMv6xjKPnRFOxG,0.19200,0.741,231964.0,0.6450,0.000000,11.0,0.1630,-5.116,0.0,0.0963,149.070,4.0,[Travis Scott],"[[pop rap, rap]]",WAKE UP,0.4880


In [122]:
3+3

6

In [52]:
for i, v in recent_50_played.iterrows():
    
    print(v['track.artists.names'])
    for i in v['track.artists.id']:
        print(spotify.artist(i)['genres'])

['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['Travis Scott']
['pop rap', 'rap']
['The Godfathers', 'Necro', 'Kool G Rap']
[]
['hardcore hip hop', 'horrorcore']
['alternative hip hop', 'boom bap', 'east coast hip hop', 'gangster rap', 'hardcore hip hop', 'hip hop', 'old school hip hop', 'queens hip hop']
['Sueco the Child']
['cali rap']
['Hot Sugar']
[]
['Gerald Albright']
['jazz funk', 'jazz saxophone', 'smooth jazz', 'smooth saxophone']
['Sueco the Child']
['cali rap']
['A$AP Ferg', 'MadeinTYO']
['hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap', 'underground hip hop']
['atl hip hop', 'hip hop', 'melodic rap', 'pop rap', 'rap', 'southern hip hop', 'trap', 'underground hip hop', 'vapor trap']
['Travis Scott', 'ROSALÍA', 'Lil Baby']
['pop ra

In [46]:
recent_50_played

,track.id,played_at,track.name,track.album.name,track.album.id,track.album.images,track.artists.names,track.artists.id
0,11kDth1aKUEUMq9r1pqyds,2020-01-10T18:29:59.199Z,5% TINT,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
1,20MuVazoNMv6xjKPnRFOxG,2020-01-10T18:26:43.090Z,WAKE UP,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
2,6NMtzpDQBTOfJwMzgMX0zl,2020-01-10T18:22:51.162Z,SKELETONS,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
3,6OaVWaGfhXn70ZJfdVcabn,2020-01-10T18:20:25.591Z,NO BYSTANDERS,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
4,1xYpVRspgaLxdJLOTQkTjg,2020-01-10T18:16:47.306Z,STOP TRYING TO BE GOD,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
5,4SZepBIPDRwPaHIjAKwRDb,2020-01-10T18:10:04.407Z,R.I.P. SCREW,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
6,2xLMifQCjDGFmkHkpNLD9h,2020-01-10T18:06:58.794Z,SICKO MODE,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
7,7KZ5MMVgBVox9ycroB2UrI,2020-01-10T18:01:45.932Z,CAROUSEL,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
8,7wBJfHzpfI3032CSD7CE2m,2020-01-10T17:58:45.831Z,STARGAZING,ASTROWORLD,41GuZcammIkupMPKH2OJ6I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Travis Scott],[0Y5tJX1MQlPlqiwlOH1tJY]
9,1gIa5W0AhvgfKgCu2RNR8C,2020-01-10T17:52:53.519Z,The City,Once Upon a Crime,04fJPhZwJpdMf2rcdSJ3o4,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[The Godfathers, Necro, Kool G Rap]","[3r6Gv2tZAgMI6djGZO4Djq, 51vjIMmHWyVqDrkxD07Jh..."


In [17]:
recent_50_played.iloc[0,]

track.id                                         11kDth1aKUEUMq9r1pqyds
played_at                                      2020-01-10T18:29:59.199Z
track.name                                                      5% TINT
track.artists                                            [Travis Scott]
track.album.name                                             ASTROWORLD
track.album.id                                   41GuZcammIkupMPKH2OJ6I
track.album.images    [{'height': 640, 'url': 'https://i.scdn.co/ima...
Name: 0, dtype: object

Key distance research: https://link.springer.com/article/10.1007/s13735-013-0036-6#Sec10

# Clustering

In [84]:
from sklearn.cluster import KMeans

In [ ]:
def song_distance_fn(x_1, x_2, w):
    artist_d = (1-m/(a+b))**2
    genre_d = (1-m/(a+b))**2
    #key_d = 
    time_signature_d = | x_1 - x_2 |
    acousticness_d = | x_1 - x_2 |
    danceability_d = | x_1 - x_2 |
    energy_d = | x_1 - x_2 |
    instrumentalness_d = | x_1 - x_2 |
    liveness_d = | x_1 - x_2 |
    loudness_d = | x_1 - x_2 |
    speechiness_d = | x_1 - x_2 |
    valence_d = | x_1 - x_2 |
    tempo_d = | x_1 - x_2 |
    
    return []

In [ ]:
def song_distance_fn_weight_wrap(x_1, x_2, w)
    return w.T * song_distance_fn(song_distance_fn(x_1, x_2))

In [118]:
X = song_features.drop(['track_artists', 'track_genres'],axis=1).set_index('track_name') 

In [119]:
kmeans = KMeans(n_clusters=10)
y = kmeans.fit_predict(X)

In [120]:
X['cluster'] = y

In [121]:
X.sort_values(by=['cluster'])

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,cluster
track_name,,,,,,,,,,,,,,
Summer Bummer (feat. A$AP Rocky & Playboi Carti),0.01550,0.760,260672.0,0.5740,0.000020,1.0,0.1990,-8.570,1.0,0.1260,134.981,4.0,0.1370,0
Quiet Storm,0.36600,0.790,265493.0,0.7280,0.000000,2.0,0.1120,-9.221,1.0,0.3910,96.978,4.0,0.4030,0
Django,0.68400,0.402,275333.0,0.2060,0.001350,5.0,0.1070,-19.784,0.0,0.0465,168.310,4.0,0.4270,0
Baltimore,0.41700,0.638,278867.0,0.2800,0.439000,9.0,0.0611,-16.516,1.0,0.0673,154.955,4.0,0.7340,0
Stars,0.82600,0.676,262640.0,0.2000,0.028900,9.0,0.1440,-17.735,0.0,0.0282,104.016,4.0,0.1760,0
Orinoco Flow,0.79700,0.685,266194.0,0.3070,0.280000,0.0,0.0988,-17.283,1.0,0.0305,114.981,4.0,0.5590,0
STARGAZING,0.00947,0.487,270715.0,0.7890,0.000004,8.0,0.1890,-3.690,1.0,0.0440,150.007,4.0,0.0564,0
Bill Collector,0.13100,0.732,171510.0,0.5570,0.174000,11.0,0.1130,-9.016,0.0,0.1150,78.458,4.0,0.3800,1
WAM,0.06240,0.838,157587.0,0.6240,0.000000,1.0,0.2170,-4.557,0.0,0.2280,95.936,4.0,0.1170,1


In [90]:
X.set_index('track_name')

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
track_name,,,,,,,,,,,,,
5% TINT,0.17300,0.712,196138.0,0.4810,0.000000,4.0,0.2410,-5.415,0.0,0.0487,140.000,4.0,0.0620
WAKE UP,0.19200,0.741,231964.0,0.6450,0.000000,11.0,0.1630,-5.116,0.0,0.0963,149.070,4.0,0.4880
SKELETONS,0.00146,0.460,145588.0,0.6860,0.000000,0.0,0.3750,-5.948,0.0,0.0367,148.054,4.0,0.2520
NO BYSTANDERS,0.02040,0.731,218284.0,0.8110,0.000000,1.0,0.4110,-3.225,1.0,0.0712,147.045,4.0,0.4260
STOP TRYING TO BE GOD,0.22100,0.588,338438.0,0.5670,0.000000,4.0,0.1350,-3.738,0.0,0.0628,150.009,4.0,0.1130
R.I.P. SCREW,0.38900,0.607,185610.0,0.6090,0.000016,0.0,0.1110,-5.978,0.0,0.2910,74.946,4.0,0.5560
SICKO MODE,0.00513,0.834,312820.0,0.7300,0.000000,8.0,0.1240,-3.714,1.0,0.2220,155.008,4.0,0.4460
5% TINT,0.17300,0.712,196138.0,0.4810,0.000000,4.0,0.2410,-5.415,0.0,0.0487,140.000,4.0,0.0620
WAKE UP,0.19200,0.741,231964.0,0.6450,0.000000,11.0,0.1630,-5.116,0.0,0.0963,149.070,4.0,0.4880


In [91]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,valence
track_id,,,,,,,,,,,,,,
11kDth1aKUEUMq9r1pqyds,0.17300,0.712,196138.0,0.4810,0.000000,4.0,0.2410,-5.415,0.0,0.0487,140.000,4.0,5% TINT,0.0620
20MuVazoNMv6xjKPnRFOxG,0.19200,0.741,231964.0,0.6450,0.000000,11.0,0.1630,-5.116,0.0,0.0963,149.070,4.0,WAKE UP,0.4880
6NMtzpDQBTOfJwMzgMX0zl,0.00146,0.460,145588.0,0.6860,0.000000,0.0,0.3750,-5.948,0.0,0.0367,148.054,4.0,SKELETONS,0.2520
6OaVWaGfhXn70ZJfdVcabn,0.02040,0.731,218284.0,0.8110,0.000000,1.0,0.4110,-3.225,1.0,0.0712,147.045,4.0,NO BYSTANDERS,0.4260
1xYpVRspgaLxdJLOTQkTjg,0.22100,0.588,338438.0,0.5670,0.000000,4.0,0.1350,-3.738,0.0,0.0628,150.009,4.0,STOP TRYING TO BE GOD,0.1130
4SZepBIPDRwPaHIjAKwRDb,0.38900,0.607,185610.0,0.6090,0.000016,0.0,0.1110,-5.978,0.0,0.2910,74.946,4.0,R.I.P. SCREW,0.5560
2xLMifQCjDGFmkHkpNLD9h,0.00513,0.834,312820.0,0.7300,0.000000,8.0,0.1240,-3.714,1.0,0.2220,155.008,4.0,SICKO MODE,0.4460
11kDth1aKUEUMq9r1pqyds,0.17300,0.712,196138.0,0.4810,0.000000,4.0,0.2410,-5.415,0.0,0.0487,140.000,4.0,5% TINT,0.0620
20MuVazoNMv6xjKPnRFOxG,0.19200,0.741,231964.0,0.6450,0.000000,11.0,0.1630,-5.116,0.0,0.0963,149.070,4.0,WAKE UP,0.4880
